In [ ]:
from IPython import display

Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Data Driven Audio Signal Processing - A Tutorial with Computational Examples

Winter Semester 2021/22 (Master Course #24512)

- lecture: https://github.com/spatialaudio/data-driven-audio-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise

Feel free to contact lecturer frank.schultz@uni-rostock.de

# Exercise 4: Singular Value Decomposition

## Objectives

- SVD
- Four subspaces in SVD domain
- Rank-1 matrix superposition

## Special Python Packages

Some convenient functions are found in `scipy.linalg`, some in `numpy.linalg` 

## Highly Recommended Material
- Kenji Hiranabe's [Graphic notes on Gilbert Strang's "Linear Algebra for Everyone"](https://github.com/kenjihiranabe/The-Art-of-Linear-Algebra/blob/main/The-Art-of-Linear-Algebra.pdf)

- Gilbert Strang's [Zoom Notes for Linear Algebra, 2021](https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/related-resources/MIT18_06SCF11_ZoomNotes.pdf) or [Lecture Notes for Linear Algebra Sample Sections](https://math.mit.edu/~gs/LectureNotes/samples.pdf)
or his [brilliant textbooks on linear algebra](http://www-math.mit.edu/~gs/)

## Some Initial Python Stuff

In [ ]:
import numpy as np
from scipy.linalg import svd, diagsvd, inv, pinv, null_space, norm
from numpy.linalg import matrix_rank

np.set_printoptions(precision=2, floatmode='fixed', suppress=True)

rng = np.random.default_rng(1234)
mean, stdev = 0, 1

# we might try out that all works for complex data as well
# for complex data the ^H operator (conj().T) needs used instead of .T only!!!
use_complex = False

## SVD of Matrix A

In [ ]:
M = 7  # number of rows
N = 4  # number of cols

# set desired rank, here N-1 for a rank deficient matrix,
# i.e. neither full column nor full row rank
rank = min(M, N) - 1
print('desired rank of A:', rank)

if use_complex:
    dtype = 'complex128'
    A = np.zeros([M, N], dtype=dtype)
    for i in range(rank):
        col = rng.normal(mean, stdev, M) + 1j*rng.normal(mean, stdev, M)
        row = rng.normal(mean, stdev, N) + 1j*rng.normal(mean, stdev, N)
        A += np.outer(col, row)  # superposition of rank-1 matrices
else:
    dtype = 'float64'
    A = np.zeros([M, N], dtype=dtype)
    for i in range(rank):
        col = rng.normal(mean, stdev, M)
        row = rng.normal(mean, stdev, N)
        A += np.outer(col, row)  # superposition of rank-1 matrices
# check if rng produced desired matrix rank
print('        rank of A:', matrix_rank(A))
print('A =\n', A)

In [ ]:
[U, s, Vh] = svd(A)
S = diagsvd(s, M, N)
V = Vh.conj().T

print('U =\n', U)
# number of non-zero sing values along diag must match rank
print('non-zero singular values: ', s[:rank])
print('S =\n', S)
print('V =\n', V)

In [ ]:
from IPython.display import Image
# image from the brilliant project Kenji Hiranabe: Graphic notes on Gilbert Strang's "Linear Algebra for Everyone"
# found at https://github.com/kenjihiranabe/The-Art-of-Linear-Algebra
# CC0-1.0 License
Image('https://github.com/kenjihiranabe/The-Art-of-Linear-Algebra/raw/main/SVD.png')

## Four Subspaces in the SVD Domain

We denote 
- column space $C(\mathbf{A})$, aka image, range
- left null space $N(\mathbf{A}^\mathrm{H})$
- row space $C(\mathbf{A}^\mathrm{H})$
- null space $N(\mathbf{A})$, aka kernel

The vectors of column space $C(\mathbf{A})$ and left null space $N(\mathbf{A}^\mathrm{H})$ are related to the matrix $\mathbf{U}$.

The vectors of row space $C(\mathbf{A}^\mathrm{H})$ and null space $N(\mathbf{A})$ are related to the matrix $\mathbf{V}$.

Since $\mathbf{U}$ and $\mathbf{V}$ are unitary matrices, they fulfill **orthonormality** and therefore the properties of orthogonal subspaces

$C(\mathbf{A}) \perp N(\mathbf{A}^\mathrm{H})$

$C(\mathbf{A}^\mathrm{H}) \perp N(\mathbf{A})$

immediately can be deduced. In other words, since we know by the property of SVD, that the matrices $\mathbf{U}$ and $\mathbf{V}$ are orthonormal, we see that

- column space is orthogonal to left null space, since both spaces are spanned by the dedicated vectors in $\mathbf{U}$
- row space is orthogonal to null space, since both spaces are spanned by the dedicated vectors in $\mathbf{V}$

In [ ]:
# image from the brilliant project Kenji Hiranabe: Graphic notes on Gilbert Strang's "Linear Algebra for Everyone"
# found at https://github.com/kenjihiranabe/The-Art-of-Linear-Algebra
# CC0-1.0 License
Image('https://github.com/kenjihiranabe/The-Art-of-Linear-Algebra/raw/main/4-Subspaces.png')

In [ ]:
# all stuff that is in matrix U
print('U =\n', U)

# Column Space C(A)
print('\ncolumn space (ortho to left null space):')
print(U[:, :rank])

# Left Null Space, if empty only 0 vector
print('left null space (ortho to column space):')
print('\t', U[:, rank:])

print('###')

# all stuff that is in matrix V
print('\nV =\n', V)

# Row Space
print('\nrow space (ortho to null space):')
print(V[:, :rank])

# Null Space N(A), if empty only 0 vector
print('null space (ortho to row space):')
print(V[:, rank:])

In [ ]:
# try null_space() function
# this very often yields the same vectors as doing the SVD manually
print('Null Space: \n', null_space(A))
# might be different from U[:,rank:], but spans the same space, we might want to check this
print('Left Null Space: \n', null_space(A.conj().T))

## Pure Row Space --> Column Space

In [ ]:
# x as linear combination of first two right singular vectors, i.e. from the row space
x = V[:, 0] + V[:, 1]
# let matrix A act on x
print(A @ x)
# must be identical with linear combination of first two left singular vectors with singular value scaling
# i.e. from the column space
print(S[0, 0] * U[:, 0] + S[1, 1] * U[:, 1])

## Row Space + Null Space --> Column Space + Null Vector

In [ ]:
# x as linear combination of row space and null space
x_r = V[:, 0]  # from row space
x_n = V[:, rank]  # from null space
x = x_r + x_n
# let matrix A act on x
print(A @ x_r)
print(A @ x)
# must be identical with linear combination of first two left singular vectors with singular value scaling
# i.e. from the column space
print(S[0, 0] * U[:, 0])
print('x from null space must yield zero vector:')
print(A @ x_n)

## Superposition of Rank-1 Matrices

Eckhart-Young theorem, cf. https://en.wikipedia.org/wiki/Low-rank_approximation

In [ ]:
Ar = np.zeros([M, N], dtype='complex128')
for r in range(rank):
    print('\nSum of', r+1, 'rank-1 matrices -> check (A-Ar)')
    Ar += S[r, r] * np.outer(U[:, r], V[:, r].conj())
    print('Frobenius norm (i.e. root(sum squared sing val)): ', norm(A-Ar, 'fro'))
    print('Spectral norm / L2 norm (i.e. sigma1)', norm(A-Ar, 2))

In the last case `Ar` is fully reconstructed from rank-1 matrices, yielding Frobenius and spectral norm for `A-Ar` of 0 (numerical precision does not give us zero).

In [ ]:
A-Ar

## Copyright

- the notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources)
- feel free to use the notebooks for your own purposes
- the text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/)
- the code of the IPython examples is licensed under under the [MIT license](https://opensource.org/licenses/MIT)
- please attribute the work as follows: *Frank Schultz, Data Driven Audio Signal Processing - A Tutorial Featuring Computational Examples, University of Rostock* ideally with relevant file(s), github URL https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise, commit number and/or version tag, year.
